# VGGT test for multipleView video


## Load the images

In [1]:
import os
def get_image_files(folder_path, extensions=['.jpg', '.jpeg', '.png', '.gif', '.bmp']):
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in extensions):
                relative_path = os.path.relpath(os.path.join(root, file), folder_path)
                full_path = os.path.join(folder_path, relative_path)
                file_paths.append(full_path)
    return file_paths
images_folder = "../collected_frames/"

if os.path.exists(images_folder):
        image_files = get_image_files(images_folder)
        print(f"\nFound {len(image_files)} Images:")
        #print(image_files)
        for img_path in image_files:
            print(img_path)
            
else:
        print(f"Folder {images_folder} doesn't exist")


Found 9 Images:
../collected_frames/cam8_frame_00001.jpg
../collected_frames/cam6_frame_00001.jpg
../collected_frames/cam5_frame_00001.jpg
../collected_frames/cam7_frame_00001.jpg
../collected_frames/cam1_frame_00001.jpg
../collected_frames/cam9_frame_00001.jpg
../collected_frames/cam2_frame_00001.jpg
../collected_frames/cam4_frame_00001.jpg
../collected_frames/cam3_frame_00001.jpg


## using the VGGT model

In [2]:
import torch
from vggt.models.vggt import VGGT
from vggt.utils.load_fn import load_and_preprocess_images

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
dtype = torch.float16


/home/ubuntu/miniconda3/envs/vggt/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


cuda


In [3]:
model = VGGT.from_pretrained("facebook/VGGT-1B").to(device) 
images = load_and_preprocess_images(image_files).to(device)

with torch.no_grad():
    with torch.cuda.amp.autocast(dtype=dtype):
        # Predict attributes including cameras, depth maps, and point maps.
        predictions = model(images)